Test of this jupyter notebook

https://github.com/BioSystemsUM/troppo/blob/master/tests/validation/TROPPO%20usage%20and%20benchmark%20notebook.ipynb

In [1]:
import sys;
# sys.path.extend(['E:\\reconstruction', 'E:\\cobamp', 'E:\\reconstruction\\src', 'E:\\cobamp\\src', 'E:/reconstruction'])
import cobra
import framed
import cobamp
import pandas as pd
import numpy as np
import pickle
import scipy as sci

# for testing the algorithms
from cobamp.wrappers import MatFormatReader
from cobamp.wrappers import COBRAModelObjectReader
from troppo.methods.imat import IMAT
from troppo.methods.gimme import GIMME
from troppo.methods.tINIT import tINIT
from troppo.methods.fastcore import FASTcore
from troppo.reconstruction_properties import FastcoreProperties, tINITProperties, GIMMEProperties, IMATProperties
from troppo.utilities.statistics import normalize, z_score

In [2]:
def load_glial_data(path):
    data = pd.read_csv(path, index_col=0)
    normalized_data = normalize(data)
    z = None

    return normalized_data, z

def get_index_core(reactions_ids, data, threshold):
    rx_names = data.index[data['0'] > threshold].tolist()
    index_core = [np.where(reactions_ids == r)[0][0] for r in rx_names]
    return index_core, rx_names

def get_index_values():
    pass

In [3]:
from project import configs
import os
generalmodelmat = os.path.join(configs.datadir,'GeneralModel.mat')

In [4]:
# mat = sci.io.loadmat('./tests/FastCoreTest.mat')['ConsistentRecon2']
model_cobra = cobra.io.load_matlab_model(generalmodelmat)

In [7]:
len(model_cobra.reactions)

10610

In [7]:
mat = sci.io.loadmat(generalmodelmat)['Teff']
#['ConsistentRecon2']

In [18]:
# dir(model_cobra)
# dir(mat)

array([[(array([[array(['AGTim'], dtype='<U5')],
       [array(['AGTix'], dtype='<U5')],
       [array(['ALAR'], dtype='<U4')],
       ...,
       [array(['biomass_reaction_Mphage'], dtype='<U23')],
       [array(['RTOT_3'], dtype='<U6')],
       [array(['ARTFR13'], dtype='<U7')]], dtype=object), <5837x10610 sparse matrix of type '<class 'numpy.float64'>'
	with 40537 stored elements in Compressed Sparse Column format>, array([[    0],
       [    0],
       [-1000],
       ...,
       [    0],
       [    0],
       [    0]], dtype=int16), array([[1000],
       [1000],
       [1000],
       ...,
       [1000],
       [1000],
       [1000]], dtype=uint16), array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]], dtype=uint8), array([[array(['10fthf5glu[c]'], dtype='<U13')],
       [array(['10fthf5glu[e]'], dtype='<U13')],
       [array(['10fthf5glu[l]'], dtype='<U13')],
       ...,
       [array(['zymst[r]'], dtype='<U8')],
       [array(['zymstnl[c]'], dtype

In [8]:
model = MatFormatReader(mat)
S = model.S
lb, ub = model.get_model_bounds(False, True)
rx_names = model.get_reaction_and_metabolite_ids()[0]

In [14]:
print(len(rx_names))
print(len(model_cobra.reactions))
exp_vector = np.random.random(len(rx_names))
# exp_vector

10610
10610


In [15]:
idx_objective

10607

In [16]:
# code for the GIMME test
idx_objective = rx_names.index('biomass_reaction_Mphage')
properties = GIMMEProperties(
    exp_vector=exp_vector,#np.array(gimme_data['0']),
    obj_frac=0.9,
    objectives= [{idx_objective:1}],
    preprocess=True,
    flux_threshold=0.9
)
algorithm = GIMME(S, lb.astype(float), ub.astype(float), properties)
model_GIMME = algorithm.run()

In [17]:
model_GIMME_final = model_cobra.copy() # this is done since it alters the original model_cobra; this way is to guarantee that a new model is changed instead of the original model
r_ids = [r.id for r in model_GIMME_final.reactions]
to_remove_ids = [r_ids[r] for r in np.where(model_GIMME==0)[0]]
model_GIMME_final.remove_reactions(to_remove_ids,True) # this is to get the ids of the reactions to be removed in the model; True is to remove the pending genes/metabolites that with the removal of the reaction can no longer be connected in the network

In [18]:
print('1\'s: ' + str(len(np.where(model_GIMME==1)[0])))
print('2\'s: ' + str(len(np.where(model_GIMME==2)[0])))
print('Genes: ' + str(len(model_GIMME_final.genes)))
print('Metabolites: ' + str(len(model_GIMME_final.metabolites)))
print('Reactions: ' + str(len(model_GIMME_final.reactions)))
print(model_GIMME_final.objective._get_expression())
print(model_GIMME_final.optimize())

1's: 1115
2's: 124
Genes: 658
Metabolites: 1857
Reactions: 1239
1.0*biomass_reaction_Mphage - 1.0*biomass_reaction_Mphage_reverse_6cff5
<Solution 0.000 at 0x1ecd55208>


In [20]:
sum(model_GIMME)

1363.0

In [35]:
from troppo.methods.gimme import GIMME
from troppo.reconstruction_properties import GIMMEProperties

import numpy as np

S = np.array([[1, -1, 0, 0, -1, 0, -1, 0, 0],
              [0, 1, -1, 0, 0, 0, 0, 0, 0],
              [0, 1, 0, 1, -1, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 1, -1, 0, 0],
              [0, 0, 0, 0, 0, 0, 1, -1, 0],
              [0, 0, 0, 0, 1, 0, 0, 1, -1]])
M, N = S.shape
lb = np.array([0] * N).astype(float)
lb[3] = -10
ub = np.array([10] * N).astype(float)

names = ['R' + str(i + 1) for i in range(N)]
exp_vector = np.random.random(N)
exp_vector[[0,2,5,8]] = -1
properties = GIMMEProperties(
    exp_vector=exp_vector,
    obj_frac=0.8,
    objectives=[np.array([0,0,0,0,0,0,0,0,1])],
    preprocess=True,
    flux_threshold=0.8
)

algorithm = GIMME(S, lb, ub, properties)

solution = algorithm.run()
print(exp_vector.round(1))
print(exp_vector > 0.5)
print(solution)
print(['R'+str(i+1) for i,v in enumerate(solution) if v > 0])

AttributeError: 'numpy.ndarray' object has no attribute 'items'

In [37]:
print(exp_vector.round(1))
print(exp_vector > 0.5)


[-1.   0.4 -1.   1.   0.6 -1.   0.2  0.1 -1. ]
[False False False  True  True False False False False]
